# 🫒 Olive Oil Export Forecasting - Data Exploration

## Project Overview

**Goal**: Build a machine learning system to predict olive oil **export volumes** based on production data and market conditions.

**Dataset**: Tunisia Olive Oil Dataset
- 7,560 records from 42 countries
- Time period: 2010-2024
- Features: Production, Exports, Prices, Dates, Countries, Seasons

**Target Variable**: `Export_Tons` (what we're predicting)

**Input Features**: 
- Production_Tons (production volume)
- USD_Price (market price)
- Country (geographical factor)
- Time features (seasonality, trends)
- Historical export patterns (lag features)

This notebook explores the data to understand patterns, distributions, and relationships that will inform our export forecasting model.

## 1️⃣ Import Libraries

In [13]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
from pathlib import Path

# Add src to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.data_loader import load_olive_oil_data, get_dataset_summary

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 2️⃣ Load Dataset

In [14]:
# Load the raw data
df = load_olive_oil_data()

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")

Loaded 7,560 olive oil production records
Date range: 2010-01-15 to 2024-09-28
Countries: 42 (Chile, South Africa, Canada, Italy, Japan...)" if df['Country'].nunique() > 5 else f"Countries: Chile, South Africa, Canada, Italy, Japan, China, Hungary, Saudi Arabia, Netherlands, Brazil, Kuwait, Czech Republic, Lebanon, Belgium, Turkey, UK, France, Greece, Sweden, Morocco, Switzerland, Argentina, Australia, Algeria, Norway, Spain, Poland, USA, Portugal, Denmark, Mexico, Romania, Egypt, Qatar, Colombia, UAE, Austria, Germany, Peru, Jordan, India, Libya
Dataset loaded: 7560 rows, 8 columns


## 3️⃣ Basic Information

In [15]:
# Display first few rows
df.head(10)

,Date,Country,Production_Tons,Export_Tons,USD_Price,Month,Year,Season
0,2013-06-28,Chile,198592,27393,15.39,6,2013,Summer
1,2019-08-26,South Africa,151287,17866,13.00,8,2019,Summer
2,2016-11-09,Canada,214355,41122,16.30,11,2016,Spring
3,2022-09-09,Italy,232414,46085,13.94,9,2022,Spring
4,2018-08-01,Japan,275341,37157,11.17,8,2018,Summer
5,2021-11-13,China,207963,47989,16.31,11,2021,Spring
6,2018-09-30,Japan,330685,30193,16.13,9,2018,Spring
7,2015-12-15,Hungary,318762,21951,7.81,12,2015,Winter
8,2013-10-26,Saudi Arabia,271805,5065,17.99,10,2013,Spring
9,2013-11-25,Netherlands,188968,30223,16.41,11,2013,Spring


In [16]:
# Data types and info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             7560 non-null   datetime64[ns]
 1   Country          7560 non-null   object        
 2   Production_Tons  7560 non-null   int64         
 3   Export_Tons      7560 non-null   int64         
 4   USD_Price        7560 non-null   float64       
 5   Month            7560 non-null   int64         
 6   Year             7560 non-null   int64         
 7   Season           7560 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 472.6+ KB


In [17]:
# Get comprehensive data info
info = get_dataset_summary(df)

print("=" * 60)
print("DATASET INFORMATION")
print("=" * 60)
print(f"\nTotal Records: {info['total_records']:,}")
print(f"\nColumns: {', '.join(df.columns.tolist())}")
print(f"\nDate Range: {info['date_range']['start'].date()} to {info['date_range']['end'].date()}")
print(f"\nNumber of Countries: {info['countries']['count']}")
print(f"Top 5 Countries: {', '.join(info['countries']['list'][:5])}")
print(f"\nProduction Stats:")
print(f"  - Total: {info['production']['total_tons']:,.0f} tons")
print(f"  - Average: {info['production']['avg_tons']:,.0f} tons")
print(f"\nExport Stats:")
print(f"  - Total: {info['exports']['total_tons']:,.0f} tons")
print(f"  - Average: {info['exports']['avg_tons']:,.0f} tons")
print(f"\nPrice Stats:")
print(f"  - Average: ${info['pricing']['avg_usd_per_ton']:.2f} per ton")
print(f"\nMissing Values:")
for col, count in info['missing_values'].items():
    if count > 0:
        print(f"  {col}: {count}")

DATASET INFORMATION

Total Records: 7,560

Columns: Date, Country, Production_Tons, Export_Tons, USD_Price, Month, Year, Season

Date Range: 2010-01-15 to 2024-09-28

Number of Countries: 42
Top 5 Countries: Algeria, Argentina, Australia, Austria, Belgium

Production Stats:
  - Total: 1,886,890,961 tons
  - Average: 249,589 tons

Export Stats:
  - Total: 198,205,193 tons
  - Average: 26,218 tons

Price Stats:
  - Average: $11.95 per ton

Missing Values:


## 4️⃣ Descriptive Statistics

In [18]:
# Numerical statistics
df.describe()

,Date,Production_Tons,Export_Tons,USD_Price,Month,Year
count,7560,7560.000000,7560.000000,7560.000000,7560.000000,7560.000000
mean,2017-05-22 23:59:59.999999744,249588.751455,26217.618122,11.952138,6.444444,2016.900000
min,2010-01-15 00:00:00,150024.000000,5005.000000,5.500000,1.000000,2010.000000
25%,2013-09-18 12:00:00,198438.000000,15375.750000,8.690000,3.750000,2013.000000
50%,2017-05-23 00:00:00,248821.000000,25148.500000,11.950000,6.000000,2017.000000
75%,2021-01-24 12:00:00,299799.000000,34404.750000,15.130000,9.000000,2021.000000
max,2024-09-28 00:00:00,349946.000000,79935.000000,18.500000,12.000000,2024.000000
std,NaN,58105.841122,14081.772041,3.721276,3.414611,4.273065


In [19]:
# Categorical statistics
print("Unique Countries:", df['Country'].nunique())
print("\nTop 10 Countries by Record Count:")
print(df['Country'].value_counts().head(10))

Unique Countries: 42

Top 10 Countries by Record Count:
Country
Chile           180
South Africa    180
Canada          180
Italy           180
Japan           180
China           180
Hungary         180
Saudi Arabia    180
Netherlands     180
Brazil          180
Name: count, dtype: int64


In [20]:
# Season distribution
print("Season Distribution:")
print(df['Season'].value_counts())

Season Distribution:
Season
Spring     2520
Summer     1932
Winter     1848
Ramadan    1260
Name: count, dtype: int64


## 5️⃣ Distribution Visualizations

In [ ]:
# Production Distribution
fig = px.histogram(
    df, 
    x='Production_Tons',
    nbins=50,
fig.show()
    title='Production Distribution',
    labels={'Production_Tons': 'Production (Tons)'},
    color_discrete_sequence=['#2E7D32']
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# Export Distribution
fig = px.histogram(
    df, 
    x='Export_Tons',
    nbins=50,
    title='Export Distribution',
    labels={'Export_Tons': 'Export (Tons)'},
    color_discrete_sequence=['#1976D2']
)
fig.show()

In [ ]:
# Price Distribution
fig = px.histogram(
    df, 
    x='USD_Price',
    nbins=50,
    title='USD Price Distribution',
    labels={'USD_Price': 'Price (USD)'},
    color_discrete_sequence=['#F57C00']
)
fig.show()

## 6️⃣ Time Series Analysis

In [ ]:
# Production over time
df_sorted = df.sort_values('Date')

fig = px.line(
    df_sorted,
    x='Date',
    y='Production_Tons',
    title='Production Over Time',
    labels={'Production_Tons': 'Production (Tons)', 'Date': 'Date'}
)
fig.show()

In [ ]:
# Monthly aggregated production
df['Date'] = pd.to_datetime(df['Date'])
monthly_production = df.groupby(df['Date'].dt.to_period('M'))['Production_Tons'].mean()
monthly_production.index = monthly_production.index.to_timestamp()

fig = px.line(
    x=monthly_production.index,
    y=monthly_production.values,
    title='Average Monthly Production',
    labels={'x': 'Month', 'y': 'Avg Production (Tons)'}
)
fig.show()

## 7️⃣ Country-wise Analysis

In [ ]:
# Top 10 countries by average production
top_countries = df.groupby('Country')['Production_Tons'].mean().sort_values(ascending=False).head(10)

fig = px.bar(
    x=top_countries.index,
    y=top_countries.values,
    title='Top 10 Countries by Average Production',
    labels={'x': 'Country', 'y': 'Avg Production (Tons)'},
    color=top_countries.values,
    color_continuous_scale='Greens'
)
fig.show()

In [ ]:
# Production by selected countries over time
selected_countries = ['Italy', 'Spain', 'Greece', 'Turkey', 'Tunisia']
df_selected = df[df['Country'].isin(selected_countries)].copy()

fig = px.line(
    df_selected.sort_values('Date'),
    x='Date',
    y='Production_Tons',
    color='Country',
    title='Production Trends by Major Producers',
    labels={'Production_Tons': 'Production (Tons)'}
)
fig.show()

## 8️⃣ Correlation Analysis

In [ ]:
# Correlation heatmap
numeric_cols = ['Production_Tons', 'Export_Tons', 'USD_Price', 'Month', 'Year']
corr_matrix = df[numeric_cols].corr()

fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    title='Correlation Heatmap',
    color_continuous_scale='RdBu_r',
    aspect='auto'
)
fig.show()

In [ ]:
# Production vs Export scatter
fig = px.scatter(
    df,
    x='Production_Tons',
    y='Export_Tons',
    color='Country',
    title='Production vs Export Volume',
    labels={'Production_Tons': 'Production (Tons)', 'Export_Tons': 'Export (Tons)'},
    opacity=0.6
)
fig.show()

## 9️⃣ Seasonal Analysis

In [ ]:
# Production by season
season_production = df.groupby('Season')['Production_Tons'].mean().sort_values(ascending=False)

fig = px.bar(
    x=season_production.index,
    y=season_production.values,
    title='Average Production by Season',
    labels={'x': 'Season', 'y': 'Avg Production (Tons)'},
    color=season_production.values,
    color_continuous_scale='Viridis'
)
fig.show()

In [ ]:
# Price by season box plot
fig = px.box(
    df,
    x='Season',
    y='USD_Price',
    title='Price Distribution by Season',
    labels={'USD_Price': 'Price (USD)'},
    color='Season'
)
fig.show()

## 📝 Key Insights

**From this exploration, we can observe:**

1. **Data Quality**: The dataset contains 7,560 comprehensive records across 42 countries (2010-2024)
2. **Production & Export Patterns**: Clear seasonal variations in olive oil production and exports
3. **Country Distribution**: Major producers include Italy, Spain, Greece, Turkey, and Tunisia
4. **Correlations**: Strong relationships between production, exports, and pricing
5. **Temporal Trends**: Long-term trends visible in the time series data
6. **Export Ratio**: Exports represent a significant portion of production across countries

**Project Focus: Export Volume Forecasting**

This project predicts **Export_Tons** (not Production_Tons) using:
- Production volumes as input feature
- Historical export patterns (lag features)
- Market conditions (USD price)
- Country-specific characteristics
- Temporal patterns (seasonality, trends)

**Next Steps:**
- Feature engineering with export lag features
- Train RandomForest model to predict Export_Tons
- Deploy via Streamlit for real-time export predictions